# <font color="maroon">Chapter 3: A Basic NMR Analysis</font>

<a href="https://githubtocolab.com/alsinmr/pyDR_tutorial/blob/main/JupyterBook/Ch3_basic_NMR.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg"></a>

In [7]:
import sys
if 'google.colab' in sys.modules:
    !git clone https://github.com/alsinmr/pyDR_tutorial.git
    

In [6]:
sys.path.append('../..')
sys.path.
import pyDR
import matplotlib.pyplot as plt
import numpy as np

nmr=pyDR.Sens.NMR()  #NMR is a sensitivity object, containing sensitivity windows for NMR experiments
nmr.new_exper(Type='R1',v0=[400,600,800],Nuc='15N') #Add R1 experiments at 3 magnetic fields
nmr.new_exper(Type='R1p',v0=600,vr=60,v1=[10,22,35,45,50],Nuc='15N') #Add R1p experiments at 5 field strengths

_=nmr.plot_Rz(norm=True)

In [5]:
colab_setup

/Users/albertsmith/Documents/GitHub/pyDR_tutorial/JupyterBook
